In [ ]:
def deduplicate_ingredients(ingredient_list):
    name_map = defaultdict(list)

    for entry in ingredient_list:
        norm_name = normalize_name(entry["name"])
        name_map[norm_name].append(entry)

    deduped = []
    for name_group in name_map.values():
        if len(name_group) == 1:
            deduped.append(name_group[0])
        else:
            deduped.append(merge_entries(name_group))

    return deduped

In [ ]:
# Fields you want to keep from each food.json record
fields_to_keep = [
    "id", "name", "name_variants", "pairings", "description",
    "taxonomy", "organism_part", "origin", "food_db_id"
]

In [ ]:
def strip_fields(entry):
    return {key: value for key, value in entry.items() if key in fields_to_keep}

In [ ]:
def clean_food_data(food_data):
    return [strip_fields(entry) for entry in food_data]

In [ ]:
with open("../data/ingredients/merged/cleaned_deduped_food.json") as f:
    raw_data = json.load(f)

In [ ]:
cleaned = clean_food_data(raw_data)
deduped = deduplicate_ingredients(cleaned)

In [ ]:
with open("../data/ingredients/merged/cleaned_deduped_food.json", "w") as f:
    json.dump(deduped, f, indent=2)